In [1]:
import cv2
import numpy as np
from skimage import metrics

# 读取灰度图
img = cv2.imread('D:/Game/upian/uling.png', 0)

# 计算图像大小
height, width = img.shape

# 定义函数：编码
def encoding(img, step):
    # 初始化编码器
    encoded = [img[0, 0]]
    predict = img[0, 0]

    # DPCM编码
    for i in range(1, height):
        for j in range(1, width):
            error = img[i, j] - predict
            if error >= step:
                predict += step
                encoded.append(step)
            elif error <= -step:
                predict -= step
                encoded.append(-step)
            else:
                encoded.append(0)
            predict += encoded[-1]

    return encoded

# 定义函数：解码
def decoding(encoded, step):
    # 初始化解码器
    decoded = np.zeros((height, width), dtype=np.uint8)
    predict = encoded[0]

    # DPCM解码
    index = 1
    for i in range(height):
        for j in range(width):
            decoded[i, j] = predict
            if index < len(encoded):
                error = encoded[index]
                if error != 0:
                    predict += error
                    predict = max(0, min(255, predict))  # 溢出保护
                index += 1

    return decoded

# 不同量化器的比特数
bits = [1, 2, 4, 8]

# 遍历各个量化器
for bit in bits:
    # 计算量化器的步长
    step = 256 / (2 ** bit)

    # DPCM编码
    encoded = encoding(img, step)

    # 量化器的比特数（队列头）
    bits_header = np.array([bit], dtype=np.uint8)

    # 对编码后的数据进行量化
    quantized = np.array(encoded, dtype=np.float32) / step
    quantized = np.round(quantized)
    quantized = np.array(quantized, dtype=np.int32)

    # 对量化后的数据进行反量化
    dequantized = quantized * step

    # DPCM解码
    decoded = decoding(dequantized, step)

    # 计算PSNR和SSIM值
    psnr = metrics.peak_signal_noise_ratio(img, decoded)
    ssim = metrics.structural_similarity(img, decoded)

    # 显示重建图像和评价指标
    cv2.imshow(f'{bit}-bit', decoded)
    print(f'{bit}-bit PSNR: {psnr:.2f}')
    print(f'{bit}-bit SSIM: {ssim:.4f}')

cv2.waitKey(0)
cv2.destroyAllWindows()

1-bit PSNR: 11.25
1-bit SSIM: 0.5624
2-bit PSNR: 11.89
2-bit SSIM: 0.5071
4-bit PSNR: 12.00
4-bit SSIM: 0.4393
8-bit PSNR: 12.68
8-bit SSIM: 0.5502


KeyboardInterrupt: 